<a href="https://colab.research.google.com/github/hojuna/Black-box_Optimization_dacon/blob/main/test_submission.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [ ]:
from google.colab import drive
drive.mount('/content/drive')

Mounted at /content/drive


In [ ]:
import pandas as pd
import numpy as np
import joblib
import os


# 모델을 저장할 디렉토리 경로 설정
# save_dir = "/content/drive/MyDrive/dacon/model/tuning_model/"


# Load the data
# train_df_org = pd.read_csv('/content/drive/MyDrive/dacon/2024_8_data/train.csv')
# test_df = pd.read_csv('/content/drive/MyDrive/dacon/2024_8_data/test.csv')

In [ ]:
import pandas as pd
from sklearn.metrics.pairwise import cosine_similarity

# 1. 데이터프레임 불러오기
# 데이터프레임이 두 개 있다고 가정하고 df1, df2로 불러온다.
df1 = pd.read_csv('/content/drive/MyDrive/dacon/2024_8_data/submission/FE6_stacking_submission.csv')  # 첫 번째 파일
df2 = pd.read_csv('/content/drive/MyDrive/dacon/2024_8_data/Fe_huber.csv')
# df2 = pd.read_csv('/content/drive/MyDrive/dacon/2024_8_data/submission/FE4_stacking_submission.csv')
# 2. 같은 id를 가진 데이터에서 y_pred값의 유사도 계산
# ID를 기준으로 데이터를 병합
merged_df = pd.merge(df1[['ID', 'y']], df2[['ID', 'y']], on='ID', suffixes=('_df1', '_df2'))

# 코사인 유사도 계산
# merged_df['similarity'] = merged_df.apply(lambda row: cosine_similarity([[row['y_df1']]], [[row['y_df2']]])[0][0], axis=1)
merged_df['similarity'] =1 - abs(merged_df['y_df1'] - merged_df['y_df2']) / max(merged_df['y_df1'].max(), merged_df['y_df2'].max())

# 3. y_pred 값을 기준으로 상위 10% id 추출
top_10_percent_df1 = df1.nlargest(int(len(df1) * 0.1), 'y')['ID'].tolist()
top_10_percent_df2 = df2.nlargest(int(len(df2) * 0.1), 'y')['ID'].tolist()

# 4. 상위 10% id들간 유사도 계산
# 상위 10%의 id를 기준으로 필터링
top_merged_df = merged_df[merged_df['ID'].isin(top_10_percent_df1) & merged_df['ID'].isin(top_10_percent_df2)]

# 5. 겹치는 id와 안 겹치는 id 리스트 반환
# 겹치는 id
common_ids = set(top_10_percent_df1).intersection(set(top_10_percent_df2))
# 안 겹치는 id
not_common_ids = set(top_10_percent_df1).symmetric_difference(set(top_10_percent_df2))

# 결과 출력
print("겹치는 ID: ", list(common_ids))
print("안 겹치는 ID: ", list(not_common_ids))

# 유사도 계산된 데이터프레임 출력
print(top_merged_df['similarity'])


# 5. not_common_ids가 어떤 df에 존재하고 어떤 df에 존재하지 않는지 확인
not_common_info = []

for id in not_common_ids:
    in_df1 = id in top_10_percent_df1
    in_df2 = id in top_10_percent_df2
    not_common_info.append({
        'id': id,
        'in_df1': in_df1,
        'in_df2': in_df2
    })

# 6. 결과를 데이터프레임으로 변환
not_common_df = pd.DataFrame(not_common_info)

# 7. 결과 출력
print("Not common IDs가 각 데이터프레임의 상위 10%에 존재하는지 여부:")
print(not_common_df)

겹치는 ID:  ['TEST_2721', 'TEST_3268', 'TEST_3511', 'TEST_4013', 'TEST_2235', 'TEST_4689', 'TEST_3561', 'TEST_3693', 'TEST_0562', 'TEST_1099', 'TEST_1018', 'TEST_4597', 'TEST_0627', 'TEST_2662', 'TEST_1009', 'TEST_1871', 'TEST_0978', 'TEST_2553', 'TEST_4335', 'TEST_0703', 'TEST_2393', 'TEST_0592', 'TEST_2103', 'TEST_0496', 'TEST_3347', 'TEST_1928', 'TEST_2060', 'TEST_3543', 'TEST_2565', 'TEST_4436', 'TEST_3480', 'TEST_0130', 'TEST_4391', 'TEST_1949', 'TEST_0783', 'TEST_4043', 'TEST_1999', 'TEST_3356', 'TEST_0809', 'TEST_3990', 'TEST_3502', 'TEST_1380', 'TEST_2249', 'TEST_1595', 'TEST_3978', 'TEST_0963', 'TEST_0608', 'TEST_0125', 'TEST_2808', 'TEST_1599', 'TEST_3209', 'TEST_0865', 'TEST_3830', 'TEST_1850', 'TEST_2765', 'TEST_1621', 'TEST_3982', 'TEST_3265', 'TEST_1089', 'TEST_0460', 'TEST_0739', 'TEST_2658', 'TEST_1156', 'TEST_2679', 'TEST_3448', 'TEST_2229', 'TEST_0864', 'TEST_4736', 'TEST_2418', 'TEST_0800', 'TEST_1898', 'TEST_2433', 'TEST_4077', 'TEST_0713', 'TEST_3065', 'TEST_1755', 'T

In [ ]:
test_df = pd.read_csv('/content/drive/MyDrive/dacon/2024_8_data/test.csv')

In [ ]:
for i in not_common_info:
  print(test_df[test_df['ID']==i["id"]])
  # print(i["ID"])

             ID       x_0       x_1       x_2       x_3     x_4       x_5  \
4033  TEST_4033  0.929017 -1.319202  1.132521  0.928131 -0.4666 -1.603482   

           x_6       x_7       x_8       x_9      x_10  
4033  0.542868 -0.068696  0.582643  0.397533 -0.017832  
             ID       x_0       x_1       x_2       x_3       x_4       x_5  \
4982  TEST_4982  0.932432 -1.193839  1.139827  0.955182 -0.471994 -1.600613   

           x_6       x_7       x_8       x_9      x_10  
4982  0.541196 -0.071568  0.583131  0.397209 -0.017859  
             ID       x_0       x_1       x_2       x_3      x_4       x_5  \
1904  TEST_1904  0.970285 -1.122419  1.109129  0.939056 -0.48429 -1.587136   

           x_6     x_7       x_8       x_9      x_10  
1904  0.545254 -0.0651  0.583269  0.400834 -0.023595  
            ID       x_0       x_1       x_2       x_3       x_4       x_5  \
493  TEST_0493  0.918469 -1.309186  1.142507  0.951061 -0.453826 -1.603024   

          x_6       x_7       x_8 

In [ ]:

'TEST_3319' in top_10_percent_df1

True

In [ ]:
test_df[test_df['ID']=='TEST_4982']

,ID,x_0,x_1,x_2,x_3,x_4,x_5,x_6,x_7,x_8,x_9,x_10
2632,TEST_2632,0.973666,-1.226481,1.14614,0.940225,-0.456507,-1.603538,0.539448,-0.074272,0.580431,0.398295,-0.017186


In [ ]:
# TEST_4711
# TEST_2220

In [ ]:
df1[df1['ID']=='TEST_2632']

,ID,y
4982,TEST_4982,90.91188


In [ ]:
df1.loc[df1['ID'] == 'TEST_2632', 'y'] = 95.91188
df1.to_csv("/content/drive/MyDrive/dacon/2024_8_data/submission/FE6_stacking_submission.csv", index=False)
